In [10]:
from monty.serialization import loadfn
from glob import glob
import time
from tqdm import tqdm
import collections
from PyAstronomy import pyasl


In [2]:
# !pip install -e /pscratch/sd/m/mavaylon/sam_ldrd/radqm9_pipeline

In [1]:
from radqm9_pipeline.modules import bucket_mol_id, train_val_test_split, build_manager

In [13]:
from radqm9_pipeline.elements import read_elements


In [15]:
# read elements
elements_dict = read_elements('/pscratch/sd/m/mavaylon/sam_ldrd/radqm9_pipeline/src/radqm9_pipeline/modules/elements.pkl')

In [11]:
# read data
base_dir = "/pscratch/sd/m/mavaylon/sam_ldrd/radqm9/updated_rad_qm9_raw/data/20230907_radQM9_trajectories"
json_files=glob(base_dir+'/*')

In [16]:
# Merge Data Files
merged_data = []
t1=time.perf_counter()

for file in tqdm(json_files):
    data = loadfn(file)
    merged_data+=data
t2=time.perf_counter()
print(t2-t1)

100%|██████████| 25/25 [31:56<00:00, 76.68s/it]

1916.9151051560184


In [54]:
i,bucket=bucket_mol_id(merged_data)

100%|██████████| 133882/133882 [00:04<00:00, 28470.57it/s]


In [6]:
temp = train_val_test_split(b, train_size=.6, val_size=.2)

100%|██████████| 1132/1132 [00:00<00:00, 19784.95it/s]


TypeError: unhashable type: 'list'

In [27]:
d=build_manager(temp)

100%|██████████| 25987/25987 [00:11<00:00, 2292.47it/s]


In [28]:
.6+.2+.2

1.0